In [ ]:
from xml.etree import ElementTree as ET
import win32con

evt_dict = {win32con.EVENTLOG_AUDIT_FAILURE: 'EVENTLOG_AUDIT_FAILURE',
            win32con.EVENTLOG_AUDIT_SUCCESS: 'EVENTLOG_AUDIT_SUCCESS',
            win32con.EVENTLOG_INFORMATION_TYPE: 'EVENTLOG_INFORMATION_TYPE',
            win32con.EVENTLOG_WARNING_TYPE: 'EVENTLOG_WARNING_TYPE',
            win32con.EVENTLOG_ERROR_TYPE: 'EVENTLOG_ERROR_TYPE'}


def parse_XML_log(event):
    tree = ET.ElementTree(ET.fromstring(event))
    root = tree.getroot()
    ns = "{http://schemas.microsoft.com/win/2004/08/events/event}"
    data = {}
    for eventID in root.findall(".//"):
        if eventID.tag == f"{ns}System":
            for e_id in eventID.iter():
                if e_id.tag == f"{ns}System":
                    pass
                elif e_id.tag == f"{ns}Provider":
                    data["Provider"] = e_id.attrib.get('Name')
                elif e_id.tag == f"{ns}TimeCreated":
                    data["TimeCreated"] = e_id.attrib.get('SystemTime')
                elif e_id.tag == f"{ns}Correlation":
                    data["ActivityID"] = e_id.attrib.get('ActivityID')
                elif e_id.tag == f"{ns}Execution":
                    data["ProcessID"] = e_id.attrib.get('ProcessID')
                    data["ThreadID"] = e_id.attrib.get('ThreadID')
                elif e_id.tag == f"{ns}EventType":
                    if not e_id.text in evt_dict.keys():
                        data['EventType'] = "unknown"
                    else:
                        data['EventType'] = str(evt_dict[e_id.text])
                else:
                    att = e_id.tag.replace(f"{ns}", "")
                    data[att] = e_id.text

        if eventID.tag == f"{ns}EventData":
            for attr in eventID.iter():
                if attr.tag == f'{ns}Data':
                    if attr.get('Name') is None:
                        data["Data"] = attr.text
                    else:
                        data[attr.get('Name')] = attr.text
                elif attr.tag == f'{ns}Binary':
                    data["Binary"] = attr.text

    return data

In [ ]:
from json import loads
from kafka import KafkaConsumer
from pymongo import MongoClient


# generating the Kafka Consumer
my_consumer = KafkaConsumer(
    'users',
    bootstrap_servers=['192.168.0.2:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)


try:
   my_client = MongoClient('127.0.0.1',27017)
   my_collection = my_client.my_application.users
   print("Connected successfully!")
except:  
   print("Could not connect to MongoDB")


for message in my_consumer:
   message = message.value
   print(type(message))
   print(message)
   # log = parse_XML_log(message)
   try:
      my_collection.insert_many(message)
      print("Data inserted successfull!")
   except:
      print("Could not insert into MongoDB!")

In [ ]:
from configparser import ConfigParser
from kafka import KafkaProducer


def read_config():
    data = {}
    config = ConfigParser()
    config.read('../kafka_config.ini')  # reading config from file
    data['topic_name'] = config.get('kafka', 'topic_name')
    data['bootstrap_servers'] = config.get('kafka', 'bootstrap_servers')
    
    return data


# producer = KafkaProducer(read_config())
kafka_config = read_config()
print(kafka_config['topic_name'])

In [16]:
import dateutil.parser

datetime_str = '2023-06-02T08:49:52.114666Z'

date1= dateutil.parser.isoparse(datetime_str)
print(date1.strftime('%Y-%m-%d %H:%M:%S'))

2023-06-02 08:49:52
